In [1]:
import json, csv
import pandas as pd
from dateutil.parser import parse
pd.__version__

'0.22.0'

In [2]:

def latlng_normalize(dataset, latlng, col_name):
    keyword = {'Latitude': 'N', 'Longitude': "W"}.get(latlng)
    for row in dataset:
        row[col_name] = float(row[col_name][:-1]) if row[col_name][-1:] is keyword else float(row[col_name][:-1]) * -1.0
        pass
    pass

def datetime_normalize(dataset, col_name):
    for row in dataset:
        row[col_name]=parse(row[col_conf['Time']['name']]).timestamp()
    pass

with open('column_config.json','r') as col_conf_file:
    col_conf = json.load(col_conf_file)
    col_conf_file.close()
    pass

print(col_conf.keys())
dataset = []

with open('input/Fireball_And_Bolide_Reports.csv','r') as csvfile:
    csvdict = csv.DictReader(csvfile)
    for a in csvdict:
        dataset.append(a)
    csvfile.close()
    pass
pd.DataFrame(dataset).head(10)

dict_keys(['RelatedData', 'Time', 'Longitude', 'Latitude'])


,Altitude (km),Calculated Total Impact Energy (kt),Date/Time - Peak Brightness (UT),Latitude (Deg),Longitude (Deg),Total Radiated Energy (J),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz
0,38,18,11/21/2009 08:53:00 PM,22.0S,29.2E,10000000000000,32.1,3,-17,-27
1,36,0.41,01/09/2015 10:41:11 AM,2.0N,28.8E,139000000000,,-10.7,-7.6,11.6
2,44,0.82,05/16/2014 12:42:48 PM,44.2S,176.2W,309000000000,,14.4,4.6,6.5
3,22.2,7.6,08/23/2014 06:29:41 AM,61.7S,132.6E,3820000000000,16.2,-2.3,5.7,16.5
4,26.3,0.11,12/12/2014 06:48:11 AM,33.5N,144.9E,33000000000,,11.5,-2.8,-2.2
5,,0.23,07/29/2014 07:38:07 AM,49.2S,172.2W,73000000000,,,,
6,28.9,0.12,06/01/2013 10:49:48 PM,65.6S,138.4E,34000000000,,,,
7,39,0.11,02/17/2015 01:19:50 PM,8.0S,11.2W,33000000000,,-28.2,3.4,4.6
8,39,0.23,10/17/2014 02:07:36 PM,4.6S,66.3W,72000000000,,,,
9,23.5,0.2,12/08/2013 03:10:09 AM,32.8N,165.1W,64000000000,,2.3,2.5,-11.3


In [3]:
for attr in list(col_conf.keys()):
    if attr == 'Latitude' or attr == 'Longitude':
        latlng_normalize(dataset, attr, col_conf[attr]['name'])
        pass
    if attr == "Time":
        datetime_normalize(dataset,col_conf[attr]['name'])
        pass
    pass
pass

In [4]:

df = pd.DataFrame(dataset)
df['Total Radiated Energy (J)'] = df['Total Radiated Energy (J)'].astype('float')
df[col_conf['Time']['name']] = df[col_conf['Time']['name']].astype('int64')
df.head(10).style.apply(lambda x: ['background: lightblue' if x.name == "Date/Time - Peak Brightness (UT)" \
                          or x.name == col_conf['Longitude']['name'] or x.name == col_conf['Latitude']['name'] \
                          else '' for i in x])


,Altitude (km),Calculated Total Impact Energy (kt),Date/Time - Peak Brightness (UT),Latitude (Deg),Longitude (Deg),Total Radiated Energy (J),Velocity (km/s),Velocity Components (km/s): vx,Velocity Components (km/s): vy,Velocity Components (km/s): vz
0,38,18,1258807980,-22,-29.2,1e+13,32.1,3,-17,-27
1,36,0.41,1420771271,2,-28.8,1.39e+11,,-10.7,-7.6,11.6
2,44,0.82,1400215368,-44.2,176.2,3.09e+11,,14.4,4.6,6.5
3,22.2,7.6,1408746581,-61.7,-132.6,3.82e+12,16.2,-2.3,5.7,16.5
4,26.3,0.11,1418338091,33.5,-144.9,3.3e+10,,11.5,-2.8,-2.2
5,,0.23,1406590687,-49.2,172.2,7.3e+10,,,,
6,28.9,0.12,1370098188,-65.6,-138.4,3.4e+10,,,,
7,39,0.11,1424150390,-8,11.2,3.3e+10,,-28.2,3.4,4.6
8,39,0.23,1413526056,-4.6,66.3,7.2e+10,,,,
9,23.5,0.2,1386443409,32.8,165.1,6.4e+10,,2.3,2.5,-11.3


In [16]:
with open('tmp/Fireball_And_Bolide_Reports.csv', 'w') as csvfile:
    csvfile.write(','.join([k for k, v in dataset[0].items()])+'\n')
    for row in dataset:
        csvfile.write(','.join([str(v) for k, v in row.items()])+'\n')